# 4. Creating and Manipulating your own Databases
**In the previous chapters, you interacted with existing databases and queried them in different ways. Now, you will learn how to build your own databases and keep them updated.**

## Creating databases and tables
It's now time to learn how to create databases and tables.

### Creating databases
Creating databases is different for every database type, and it often requires the use of a command line tool or management application. All these tools are beyond the scope of this class. However with SQLite, if you supply a filename that does not exist to the create_engine method, it will create that file which creates the database.

### Building a table
In this example, we are creating a table to store data about our `employees`. We've already created an engine and metadata. We want to store the `employee`'s `id`, `name`, `salary`, and `active` status. 

We start by importing everything we need. This includes the Table and Column objects along with any SQL Data types we want to use in our columns. There are many SQL Data types, and the SQLAlchemy documentation is a great place to see all the ones that are available. Next, we use the Table object to create a table named employees, in our metadata with a few columns. The first column is the id column and we made it an integer type to store the employees id. Next we have the name column which is a String that can be up to 255 characters long. Next we have the salary column which is a decimal type to hold the employees yearly salary. The last column is the active column which is a Boolean to let us know if they are still actively employed. We store that table object as employees. Now, we use the `create_all` method on the metadata objects and pass it the engine to create the table in the database itself. Finally, we use the `table_names` method of the engine to verify that the table was created.

In [28]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Numeric, Boolean

engine = create_engine('sqlite:///')
metadata = MetaData()

employees = Table('employees', metadata,
                  Column('id', Integer()),
                  Column('name', String(255)),
                  Column('salary', Numeric()),
                  Column('active', Boolean()))
metadata.create_all(engine)
engine.table_names()   

['employees']


### Creating tables
As you saw in the prior example, creating tables can be done with SQLAlchemy just like queries. We will still use the Table object similar to how we used it for reflection. We'll just remove the `autoload` keyword arguments and replace them with the Column objects we want to exist in the table. Once we have our table object, we can use the metadata's `create_all` method and the engine to create the table in the actual database. While it's easy to create tables with SQLAlchemy, if you want to make changes to an existing table structure, such as adding or removing columns, you'll need to build raw SQL ALTER statements or use a tool like Alembic, which is outside the scope of this course.

### Creating tables - additional column options
In addition to Columns having a type, they can also have constraints and defaults that are additional keyword arguments to the Column object. With constraints we can require that a column be `unique` or specify that a column can not be empty or null. There other more complex constraints as well that can be used to require any Boolean condition you can imagine. `Default` sets the initial value of a field if one is not supplied during an insert statement. Let's create a table with some constraints.

### Building a table with additional options
We're going to create the same employees table; however, we want to make sure the name column is unique and not allowed to be empty. We established a default salary of 100.00, and also set active to be True by default. Finally, we check to see what constraints are on the table using the constraints attribute. We can see that our desired constraints and defaults are in place.


In [63]:
engine = create_engine('sqlite:///')
metadata = MetaData()
employees = Table('employees', metadata,
                  Column('id', Integer()),
                  Column('name', String(255), unique=True, nullable=False),
                  Column('salary', Numeric(), default=100.00),
                  Column('active', Boolean(), default=True))
metadata.create_all(engine)
employees.constraints

{CheckConstraint(<sqlalchemy.sql.elements.BinaryExpression object at 0x0000015E2F07FE80>, name='_unnamed_', table=Table('employees', MetaData(bind=None), Column('id', Integer(), table=<employees>), Column('name', String(length=255), table=<employees>, nullable=False), Column('salary', Numeric(), table=<employees>, default=ColumnDefault(100.0)), Column('active', Boolean(), table=<employees>, default=ColumnDefault(True)), schema=None), _create_rule=<sqlalchemy.util.langhelpers.portable_instancemethod object at 0x0000015E2F007C80>, _type_bound=True),
 PrimaryKeyConstraint(),
 UniqueConstraint(Column('name', String(length=255), table=<employees>, nullable=False))}

## Creating tables with SQLAlchemy
Previously, you used the `Table` object to reflect a table from an *existing* database, but what if you wanted to create a *new* table? You'd still use the `Table` object; however, you'd need to replace the `autoload` and `autoload_with` parameters with Column objects.

The `Column` object takes a name, a SQLAlchemy type with an optional format, and optional keyword arguments for different constraints.

When defining the table, recall how to pass in `255` as the maximum length of a String by using `Column('name', String(255))`.

After defining the table, you can create the table in the database by using the `.create_all()` method on metadata and supplying the engine as the only parameter.

- Import `Table`, `Column`, `String`, `Integer`, `Float`, `Boolean` from `sqlalchemy`.
- Build a new table called `data` with columns `'name'` (`String(255)`), `'count'` (`Integer()`), `'amount'`(`Float()`), and `'valid'` (`Boolean()`) columns. The second argument of `Table()` needs to be `metadata`.
- Create the table in the database by passing `engine` to `metadata.create_all()`.

In [2]:
# Import create_engine, MetaData, and Table
from sqlalchemy import create_engine, MetaData

# Create engine: engine
engine = create_engine('sqlite:///')

# Create a metadata object: metadata
metadata = MetaData()

In [3]:
# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

# Define a new table with a name, count, amount, and valid column: data
data = Table('data', metadata,
             Column('name', String(255)),
             Column('count', Integer()),
             Column('amount', Float()),
             Column('valid', Boolean())
)

# Use the metadata to create the table
metadata.create_all(engine)

# Print table details
print(repr(data))

Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>), schema=None)


## Constraints and data defaults
You're now going to practice creating a table with some constraints. Often, you'll need to make sure that a column is unique, nullable, a positive value, or related to a column in another table. This is where constraints come in.

In addition to constraints, you can also set a default value for the column if no data is passed to it via the `default` keyword on the column.

- Build a new table called data with a unique `name` (String), `count` (Integer) defaulted to `1`, `amount` (Float), and `valid` (Boolean) defaulted to `False`.
- Create the table in the database and to print the table details for `data`.

In [15]:
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

metadata.drop_all(engine)
engine.table_names()

[]

In [46]:
# Create engine: engine
engine = create_engine('sqlite:///')

# Create a metadata object: metadata
metadata = MetaData()

In [47]:
# Define a new table with a name, count, amount, and valid column: data
data = Table('data', metadata,
             Column('name', String(255), unique=True),
             Column('count', Integer(), default=1),
             Column('amount', Float()),
             Column('valid', Boolean(), default=False)
            )

# Use the metadata to create the table
metadata.create_all(engine)

# Print the table details
print(repr(metadata.tables['data']))

Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>, default=ColumnDefault(1)), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>, default=ColumnDefault(False)), schema=None)


---
## Inserting data into a table
With our table created, we are ready to insert some data into the table.

### Adding data to a table
We do this with an `insert()` statement. `insert()` takes a table name as an argument, and then all the values we want to insert are added in a `values` clause as `column=value` pairs. The insert doesn't return any rows, so we don't need to use a fetch method after executing the statement. Let's insert a record into our employees table.

### Inserting one row
After we've created the engine, established the connection, and created or reflected the table, we import the insert statement from sqlalchemy. Next, we build an insert statement for the employees table with the following values. We'll set the id column to `1`, and the name column to Jason. We'll give Jason a salary of 1 and make him an active employee. With our insert statement built, we can now execute the statement and store the result proxy it returns. Remember the insert statement doesn't return any rows; however, we can use the `rowcount` attribute of the result proxy to see how many rows where inserted.


In [64]:
from sqlalchemy import insert

stmt = insert(employees).values(id=1, name='Jason',
                               salary=1.00, active=True)
connection = engine.connect()
result_proxy = connection.execute(stmt)
print(result_proxy.rowcount)

1


### Inserting multiple rows
It is also possible to insert multiple records at one with an insert statement by building an insert statement without the values clause. Then we build a list of dictionaries to represent our column=value pairs for each record. Finally, we pass both the insert statement and the list of dictionaries to the connection's execute method which will insert all the records in the list. Let's add some more employees with a multiple insert statement.

We start by building an insert statement for the employees table without a values clause. Next, we build a list of dictionaries, one dictionary for each record we want to insert. I typically call this list `values_list` to make its purpose clear. Each dictionary has the columns and associated values we want to insert. We pass both the insert statement and the list of dictionaries to the execute method on connection. Again there is no need to call a fetch method since an insert returns no rows. Finally, we use the `rowcount` attribute of the result proxy to check how many records were inserted.

In [65]:
stmt = insert(employees)
values_list = [{'id':2, 'name': 'Rebecca',
               'salary': 2.00, 'active': True},
              {'id': 3, 'name': 'Bob',
              'salary': 0.00, 'active': False}]
result_proxy = connection.execute(stmt, values_list)
print(result_proxy.rowcount)

2


## Inserting a single row
There are several ways to perform an insert with SQLAlchemy; however, we are going to focus on the one that follows the same pattern as the `select` statement.

It uses an `insert` statement where you specify the table as an argument, and supply the data you wish to insert into the value via the `.values()` method as keyword arguments. For example, if `my_table` contains columns `my_col_1` and `my_col_2`, then `insert(my_table).values(my_col_1=5, my_col_2="Example")` will create a row in `my_table` with the value in `my_col_1` equal to 5 and value in `my_col_2` equal to `"Example"`.

Notice the difference in syntax: when appending a `where` statement to an existing statement, we include *the name of the table* as well as the name of the column, for example `new_stmt = old_stmt.where(my_tbl.columns.my_col == 15)`. This is necessary because the existing statement might involve several tables.

On the other hand, you can only `insert` a record into a single table, so you do not need to include the name of the table when using `values()` to insert, e.g. `stmt = insert(my_table).values(my_col = 10)`.

Here, the name of the table is `data`. You can run `repr(data)` in the console to examine the structure of the table.

- Import `insert` and `select` from the `sqlalchemy` module.
- Build an insert statement `insert_stmt` for the `data` table to set `name` to `'Anna'`, `count` to `1`, `amount` to `1000.00`, and `valid` to `True`.
- Execute `insert_stmt` with the `connection` and store the `results`.
- Print the `.rowcount` attribute of `results` to see how many records were inserted.
- Build a select statement to query `data` for the record with the `name` of `'Anna'`.
- Print the results of executing the select statement.

In [50]:
# Import insert and select from sqlalchemy
from sqlalchemy import insert, select

# Build an insert statement to insert a record into the data table: insert_stmt
insert_stmt = insert(data).values(name='Anna', 
                                  count=1, amount=1000.00, valid=True)

# Execute the insert statement via the connection: results
connection = engine.connect()
results = connection.execute(insert_stmt)

# Print result rowcount
print(results.rowcount)

1


In [51]:
# Build a select statement to validate the insert: select_stmt
select_stmt = select([data]).where(data.columns.name == 'Anna')

# Print the result of executing the query.
print(connection.execute(select_stmt).first())

('Anna', 1, 1000.0, True)


## Inserting multiple records at once
It's time to practice inserting multiple records at once!

When inserting multiple records at once, you do not use the `.values()` method. Instead, you'll want to first build a list of dictionaries that represents the data you want to insert, with keys being the names of the columns. in the `.execute()` method, you can pair this list of dictionaries with an `insert` statement, which will insert all the records in your list of dictionaries.

- Build a list of dictionaries called `values_list` with two dictionaries. In the first dictionary set `name` to `'James'`, `count` to `1`, `amount` to `800.00`, and `valid` to `True`. In the second dictionary of the list, set `name` to `'Taylor'`, `count` to `1`, `amount` to `750.00`, and `valid` to `False`.
- Build an `insert` statement for the `data` table for a multiple insert, save it as `stmt`.
- Execute `stmt` with the `values_list` via `connection` and store the `results`. Make sure `values_list` is the second argument to `.execute()`.
- Print the `rowcount` of the `results`.

In [52]:
# Build a list of dictionaries: values_list
values_list = [
    {'name': 'James', 'count': 1, 'amount': 800.00, 'valid': True},
    {'name': 'Taylor', 'count': 1, 'amount': 750.00, 'valid': False}
]

# Build an insert statement for the data table: stmt
stmt = insert(data)

# Execute stmt with the values_list: results
results = connection.execute(stmt, values_list)

# Print rowcount
print(results.rowcount)

2


## Loading a CSV into a table
You've done a great job so far at inserting data into tables! You're now going to learn how to load the contents of a CSV file into a table.

One way to do that would be to read a CSV file line by line, create a dictionary from each line, and then use `insert()`, like you did in the previous exercise.

But there is a faster way using `pandas`. You can read a CSV file into a DataFrame using the `read_csv()` function (this function should be familiar to you, but you can run `help(pd.read_csv)` in the console to refresh your memory!). Then, you can call the [`.to_sql()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) method on the DataFrame to load it into a SQL table in a database. The columns of the DataFrame should match the columns of the SQL table.

`.to_sql()` has many parameters, but in this exercise we will use the following:
- `name` is the name of the SQL table (as a string).
- `con` is the connection to the database that you will use to upload the data.
- `if_exists` specifies how to behave if the table already exists in the database; possible values are `"fail"`, `"replace"`, and `"append"`.
- `index` (`True` or `False`) specifies whether to write the DataFrame's index as a column.

In this exercise, you will upload the data contained in the `census.csv` file into an existing table `"census"`. The `connection` to the database has already been created for you.

- Use `pd.read_csv()` to load the `"census.csv" `file into a DataFrame. Set the `header` parameter to `None` since the file doesn't have a header row.
- Rename the columns of `census_df` to `"state"`, `"sex"`, `"age"`, `"pop2000"`, and `"pop2008"` to match the columns of the `"census"` table in the database.

In [58]:
# import pandas
import pandas as pd

# read census.csv into a DataFrame : census_df
census_df = pd.read_csv('census.csv', header=None)

# rename the columns of the census DataFrame
census_df.columns = ['state', 'sex', 'age', 'pop2000', 'pop2008']

- Use the `.to_sql()` method on `census_df` to **append** the data to the `"census"` table in the database using the `connection`.

- Since `"census"` already exists in the database, you will need to specify an appropriate value for the `if_exists` parameter.

In [72]:
# import pandas
import pandas as pd

# read census.csv into a DataFrame : census_df
census_df = pd.read_csv("census.csv", header=None)

# rename the columns of the census DataFrame
census_df.columns = ['state', 'sex', 'age', 'pop2000', 'pop2008']

# append the data from census_df to the "census" table via connection
census_df.to_sql(name='census', con=connection, 
                 if_exists='append', index=False)

*The* `pandas` *package provides us with an efficient way to load a DataFrames into a SQL table. If you would create and execute a statement to select all records* `from` *census, you would see that there are 8772 rows in the table.*

In [74]:
stmt = 'SELECT COUNT(*) FROM census'
result_proxy = connection.execute(stmt)
results = result_proxy.fetchall()
print(results)

[(8772,)]


---
## Updating data in a table
It's also possible to edit update data in a database.

### Updating data in a table
This is done using the `update` statement. The update statement works just like an `insert`, but it has an additional `where` clause to determine which records will be updated. The `values` clause contains only the `column=value` pairs we want to change.

### Updating one row
In this example, I want to make the employee with the `ID` of `3` an active employee. I start by importing the update statement from sqlalchemy. Next I build an update statement for the employees table that, uses a `where` clause to target employee 3 and then in the values clause sets them active. Next I execute the statement and print the `rowcount` to make sure that only 1 employee was updated.


In [66]:
from sqlalchemy import update
stmt = update(employees)
stmt = stmt.where(employees.columns.id == 3)
stmt = stmt.values(active=True)
result_proxy = connection.execute(stmt)
print(result_proxy.rowcount)

1



### Updating multiple rows
It's possible to update multiple records by having a `where` clause that would target multiple records. Because this is so easy to do, I always advise you to check the rowcount and make sure the right number of records were updated.

### Inserting multiple rows
In this example, we need to set all active employees to be inactive and with a salary of 0. We start by building an update statement for the employees table with a where clause that matches all active employees. Then we update the active and salary columns with the changes. Next we execute the statement and finally print the rowcount to make sure I updated the proper number of employees.

In [67]:
stmt = update(employees)
stmt = stmt.where(employees.columns.active == True)
stmt = stmt.values(active=False, salary=0.00)
result_proxy = connection.execute(stmt)
print(result_proxy.rowcount)

3


### Correlated updates
We have already restored all the employees back to the prior active status and previous salaries, and now we want to pay all our employees the same amount.

We start by building a select statement to select the maximum salary we currently pay any employee. Next we build an update statement for the employees table **without** a `where` clause so it will update every record. In the values clause we set the salary column to the select statement. When we execute the statement, it will find the maximum salary in the table using the `new_salary` select statement and that maximum salary will be used as the value for the salary field in the update statement that affects every record in the table. We can see when we print the `rowcount` that it affected all three employees.


In [69]:
from sqlalchemy import desc
new_salary = select([employees.columns.salary])
new_salary = new_salary.order_by(
        desc(employees.columns.salary))
new_salary = new_salary.limit(1)
stmt = update(employees)
stmt = stmt.values(salary=new_salary)
result_proxy = connection.execute(stmt)
print(result_proxy.rowcount)

3


When you use a `select` statement to get the value to be used in an update statement as we did in the previous example, this is called a Correlated Update. A correlated update is often used to select data from another table or the maximum value of a column to use as the value of the update.

## Updating individual records
The `update` statement is very similar to an `insert` statement. For example, you can update all wages in the `employees` table as follows:
```python
stmt = update(employees).values(wage=100.00)
```
The `update` statement also typically uses a `where` clause to help us determine what data to update. For example, to only update the record for the employee with ID 15, you would append the previous statement as follows:
```python
stmt = stmt.where(employees.id == 15)
```
You'll be using the FIPS state code here, which is appropriated by the U.S. government to identify U.S. states and certain other associated areas.

For your convenience, the names of the tables and columns of interest in this exercise are: `state_fact` (Table), `name` (Column), and `fips_state` (Column).

- Build a statement to *select* all columns from the `state_fact` table where the value in the `name` column is `'New York'`. Call it `select_stmt`.
- Fetch all the results and assign them to `results`.
- Print the results and the `fips_state` column of the first row of the results.

```python
# Build a select statement: select_stmt
select_stmt = select([state_fact]).where(state_fact.columns.name == 'New York')

# Execute select_stmt and fetch the results
results = connection.execute(select_stmt).fetchall()

# Print the results of executing the select_stmt
print(results)

# Print the FIPS code for the first row of the result
print(results[0]['fips_state'])
```

```
[('32', 'New York', 'NY', 'USA', 'state', '10', 'current', 'occupied', '', '0', 'N.Y.', 'II', '1', 'Northeast', '2', 'Mid-Atlantic', '2')]

0
```

- Notice that there is only one record in `state_fact` for the state of New York. It currently has the FIPS code of `0`.
    - Build an `update` statement to change the `fips_state` column code to `36`, save it as `update_stmt`.
    - Use a `where` clause to filter for states with the `name` of `'New York'` in the `state_fact` table.
    - Execute `update_stmt` via the `connection` and save the output as `update_results`.

```python
# Build a statement to update the fips_state to 36: update_stmt
update_stmt = update(state_fact).values(fips_state = 36)

# Append a where clause to limit it to records for New York state
update_stmt = update_stmt.where(state_fact.columns.name == 'New York')

# Execute the update statement: update_results
update_results = connection.execute(update_stmt)
```

- Now you will confirm that the record for New York was updated by selecting all the records for New York from `state_fact` and repeating what you did before.
    - Execute `select_stmt` again, fetch all the results, and assign them to `new_results`. Print the `new_results` and the `fips_state` column of the first row of the `new_results`.

```python
# Execute select_stmt again and fetch the new results
new_results = connection.execute(select_stmt).fetchall()

# Print the new_results
print(new_results)

# Print the FIPS code for the first row of the new_results
print(new_results[0]['fips_state'])
```

```
[('32', 'New York', 'NY', 'USA', 'state', '10', 'current', 'occupied', '', '36', 'N.Y.', 'II', '1', 'Northeast', '2', 'Mid-Atlantic', '2')]

36
```

## Updating multiple records
As Jason discussed in the video, by using a `where` clause that selects more records, you can update multiple records at once. Unlike inserting, updating multiple records works exactly the same way as updating a single record (as long as you are updating them with the same value). It's time now to practice this.

For your convenience, the names of the tables and columns of interest in this exercise are: `state_fact` (Table), `notes` (Column), and `census_region_name` (Column).

- Build an `update` statement to update the `notes` column in the `state_fact` table to `'The Wild West'`. Save it as `stmt`.
- Use a `where` clause to filter for records that have `'West'` in the `census_region_name` column of the `state_fact` table.
- Execute `stmt_west` via the `connection` and save the output as `results`.
- Print `rowcount` of the `results`.

```python
# Build a statement to update the notes to 'The Wild West': stmt
stmt = update(state_fact).values(notes='The Wild West')

# Append a where clause to match the West census region records: stmt_west
stmt_west = stmt.where(state_fact.columns.census_region_name == 'West')

# Execute the statement: results
results = connection.execute(stmt_west)

# Print rowcount
print(results.rowcount)
```
```

13
```

## Correlated updates
You can also update records with data from a select statement. This is called a correlated update. It works by defining a `select` statement that returns the value you want to update the record with and assigning that select statement as the value in `update`.

You'll be using a `flat_census` in this exercise as the target of your correlated update. The `flat_census` table is a summarized copy of your census table, and contains, in particular, the `fips_state` columns.

- Build a statement to select the `name` column from `state_fact`. Save the statement as `fips_stmt`.
- Append a where clause to `fips_stmt` that matches `fips_state` from the `state_fact` table with `fips_code` in the `flat_census` table.
- Build an update statement to set the `state_name` in `flat_census` to `fips_stmt`. Save the statement as `update_stmt`.
- Execute `update_stmt`, store the `results` and print the `rowcount` of `results`.

```python
# Build a statement to select name from state_fact: fips_stmt
fips_stmt = select([state_fact.columns.name])

# Append a where clause to match the fips_state to flat_census fips_code: fips_stmt
fips_stmt = fips_stmt.where(state_fact.columns.fips_state ==
                            flat_census.columns.fips_code)

# Build an update statement to set the name to fips_stmt_where: update_stmt
update_stmt = update(flat_census).values(state_name=fips_stmt)

# Execute update_stmt: results
results = connection.execute(update_stmt)

# Print rowcount
print(results.rowcount)
```
```

51
```

---
## Deleting data from a database
###  Deleting data from a table
First to delete data from a table, we use the `delete()` statement. The `delete()` statement targets a table and uses a `where()` clause to determine which rows to delete. It's not simple or fast to restore large databases or tables if you delete them so be very cautious when using the delete statement.

### Deleting all data from a table
In this example, we are going to delete all the data from a table named `extra_employees`. We start by importing the delete statement. next we're going to build a select statement to count the records in the `extra_employees` table in order to make sure that we delete the correct number of records. We execute the statement and use the scalar fetch method to get just the number of records back. 

```python
from sqlalchemy import delete
stmt = select([func.count(extra_employees.columns.id)])
connection.execute(stmt).scalar()
```
```

3
```

Then we build a `delete()` statement that targets the `extra_employees` table without a `where` clause. Now we execute the `delete()` statement and check the row count to make sure it matches the count from our `select()` statement.
```python
delete_stmt = delete(extra_employees)
result_proxy = connection.execute(delete_stmt)
result_proxy.rowcount
```
```

3
```

### Deleting specific rows
We can delete specific rows by using a where clause on the `delete()` statement. Remember much like update, it's import to check the rowcount to make sure you didn't accidentally delete too many rows.

Here we are removing employee 3 from the employees table. We begin by building a `delete()` statement for the `employees` table and use a `where` clause to target the employee with the ID of 3. Then we execute the statement and check the row count to make sure we only removed one record.
```python
stmt = delete(employees).where(employees.columns.id == 3)
result_proxy = connection.execute(stmt)
result_proxy.rowcount
```
```

1
```

### Dropping a table completely
Sometimes, you need to delete the table itself from the database, this is done with the `drop()` method on the table. We pass the engine in as an argument to specify which database we are dropping the table from. Dropping a table only removes it and the data it holds from the database, and does not remove it from metadata or our table object.

### Dropping a table
In this example, we are going to drop the `extra_employees` table. We call the drop method on the table and pass it the engine that points to our database. Finally, we verify that it was deleted by using the exists method with the engine to check if it is still there.
```python
extra_employees.drop(engine)
print(extra_employees.exists(engine)
```
```

False
```

### Dropping all the tables
It's also possible to drop all the tables in a database by using the `drop_all()` method on the metadata object. It works exactly the same as the drop method, except it affects all tables in the database.

So to completely empty the database we are using, we call the `drop_all()` method and pass in the engine to specify which database we are working on. Finally when we check the `table_names` on the engine object, we can see that there are no more tables.
```python
metadata.drop_all(engine)
engine.table_names()
```
```

[]
```

## Deleting all the records from a table
Often, you'll need to empty a table of all of its records so you can reload the data. You can do this with a `delete` statement with just the table as an argument. For example, the table `extra_employees` is deleted by executing as follows:
```python
delete_stmt = delete(extra_employees)
result_proxy = connection.execute(delete_stmt)
```
Do be careful, though, as deleting cannot be undone.

- Import `delete` and `select` from sqlalchemy.
- Build a `delete` statement to remove all the data from the `census` table. Save it as `delete_stmt`.
- Execute `delete_stmt` via the `connection` and save the `results`.
- `Select` all remaining rows from the `census` table and print the result to confirm that the table is now empty.

```python
# Import delete, select
from sqlalchemy import delete, select

# Build a statement to empty the census table: stmt
delete_stmt = delete(census)

# Execute the statement: results
results = connection.execute(delete_stmt)

# Print affected rowcount
print(results.rowcount)

# Build a statement to select all records from the census table : select_stmt
select_stmt = select([census])

# Print the results of executing the statement to verify there are no rows
print(connection.execute(select_stmt).fetchall())
```
```

8772

[]
```

## Deleting specific records
By using a `where()` clause, you can target the `delete` statement to remove only certain records. For example, all rows from the `employees` table that had `id` 3 is deleted with the following delete statement:
```python
delete(employees).where(employees.columns.id == 3) 
```
Here you'll delete ALL rows which have `'M'` in the `sex` column and `36` in the `age` column. We have included code at the start which computes the total number of these rows. It is important to make sure that this is the number of rows that you actually delete.

- Build a `delete` statement to remove data from the `census` table. Save it as `delete_stmt`.
- Append a `where` clause to `delete_stmt` that contains an `and_` to filter for rows which have `'M'` in the `sex` column **AND** `36` in the `age` column.
- Execute the delete statement.
- Print the `rowcount` of the `results`, as well as `to_delete`, which returns the number of rows that should be deleted. These should match and this is an important sanity check.

```python
# Build a statement to count records using the sex column for Men ('M') age 36: count_stmt
count_stmt = select([func.count(census.columns.sex)]).where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the select statement and use the scalar() fetch method to save the record count
to_delete = connection.execute(count_stmt).scalar()

# Build a statement to delete records from the census table: delete_stmt
delete_stmt = delete(census)

# Append a where clause to target Men ('M') age 36: delete_stmt
delete_stmt = delete_stmt.where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the statement: results
results = connection.execute(delete_stmt)

# Print affected rowcount and to_delete record count, make sure they match
print(results.rowcount, to_delete)
```
```

51 51
```

## Deleting a table completely
You're now going to practice dropping individual tables from a database with the `.drop()` method, as well as *all* tables in a database with the `.drop_all()` method.

As Spider-Man's Uncle Ben said: With great power, comes great responsibility. Do be careful when deleting tables, as it's not simple or fast to restore large databases! Remember, you can check to see if a table exists on an `engine` with the `.exists(engine)` method.

- Drop the `state_fact` table by applying the method `.drop()` to it and passing it the argument `engine` *(in fact, `engine` will be the sole argument for **every** function/method in this exercise.)*
- Check to see if `state_fact` exists via `print`. Use the `.exists()` method with `engine` as the argument.
- Drop all the tables via the `metadata` using the `.drop_all()` method.
- Use a print statement to check if the `census` table exists.

```python
# Drop the state_fact table
state_fact.drop(engine)

# Check to see if state_fact exists
print(state_fact.exists(engine))
```
```

False

```
```python
# Drop all tables
metadata.drop_all(engine)

# Check to see if census exists
print(engine.table_names())
```
```

[]
```